In [21]:
import pandas as pd
import numpy as np
import math

main = pd.read_csv('combined_data.csv')
main['year'] = main['month'].map(lambda x: int('20'+x[3:]))

main_state = pd.read_csv('all_state_trips_lag.csv')
main_state_month = pd.read_csv('combined_state_data.csv')
state_month_mode = pd.read_csv('combined_state_mode_data.csv')

In [22]:
yrs = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016
      ,2017, 2018]

cities = {'Boston': 'Boston city, Massachusetts', 'Chicago': 'Chicago city, Illinois', 
          'Cleveland':'Cleveland city, Ohio', 'Denver':'Denver city, Colorado', 'Houston':'Houston city, Texas', 
         'LA':'Los Angeles city, California', 'Miami':'Miami city, Florida', 'NYC':'New York city, New York', 
          'San Francisco':'San Francisco city, California', 'Seattle':'Seattle city, Washington'}

In [23]:
def unfuck_columns(df):
    inc = [s for s in df.columns if "Adjusted Dollars" in s]
    df = df.rename(columns = {inc[0]:'Median Household Income'})
    if 'Average Commute to Work (In Min)' in df.columns:
        df = df.rename(columns = {'Average Commute to Work (In Min)':'Average Commute to Work (in Min)'})
    if 'Average Commute to Work (in min)' in df.columns:
        df = df.rename(columns = {'Average Commute to Work (in min)':'Average Commute to Work (in Min)'})
    if 'Car, truck, or van' in df.columns:
        df = df.rename(columns = {'Car, truck, or van':'Car, Truck, or Van'})
    if 'Other means' in df.columns:
        df = df.rename(columns = {'Other means':'Other Means'})
    if 'Public transportation (Includes Taxicab)' in df.columns:
        df = df.rename(columns = {'Public transportation (Includes Taxicab)':'Public Transportation (Includes Taxicab)'})
    if 'Worked at home' in df.columns:
        df = df.rename(columns = {'Worked at home':'Worked At Home'})
    
    return df
    

def city_yr_stats(city, year, is_state,name_dict, and_month=False, and_mode=False):
    if is_state:
        file = 'State_ACS/'+str(year)+'.csv'
    else:
        file = 'ACS/'+str(year)+'.csv'
    df = pd.read_csv(file, index_col=0, header=1, encoding='unicode_escape')
    df = df.dropna(how='all')
    df = df.fillna(-1)
    
    Lname = name_dict[city]
    c_loc = df.columns.get_loc(Lname)
    cit = df.iloc[:,[c_loc,c_loc+1]]
    cit.columns = ['extra' if 'Unnamed' in x else x for x in cit.columns]
    cit['extra'] = cit['extra'].str.rstrip('%').astype('float') / 100.0
    cit = cit.fillna(-1)
    cit[Lname] = np.where(cit.extra != -1,cit['extra'] ,cit[Lname])
    cit = cit.drop(columns='extra')
    cit= cit.T
    if is_state:
        if and_month:
            cit['state'] = city
        else:
            cit['State'] = city
    else:
        cit['city'] = city
    cit['year'] = year
    
    if is_state:
        if and_month:
            if and_mode:
                samp = state_month_mode[(state_month_mode.state==city) & (state_month_mode.year==year)]
                res = pd.merge(cit, samp, on=['state','year'], sort=True)
                res = res.loc[:,~res.columns.duplicated()]
                res1 = unfuck_columns(res)
            else:
                samp = main_state_month[(main_state_month.state==city) & (main_state_month.year==year)]
                res = pd.merge(cit, samp, on=['state','year'], sort=True)
                res = res.loc[:,~res.columns.duplicated()]
                res1 = unfuck_columns(res)
                
        else:
            samp = main_state[(main_state.State==city) & (main_state.year==year)]
            res = pd.merge(cit, samp, on=['State','year'], sort=True)
            res = res.loc[:,~res.columns.duplicated()]
            res1 = unfuck_columns(res)
        
    else:
        samp = main[(main.city==city) & (main.year==year)]
        res = pd.merge(cit, samp, on=['city','year'], sort=True)
        res = res.loc[:,~res.columns.duplicated()]
        res1 = unfuck_columns(res)

    return res1


In [24]:
test0 = city_yr_stats('Boston', 2017, False, cities)
full_df = pd.DataFrame(columns = test0.columns)

C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
for city in cities:
    for yr in yrs:
        cur_df = city_yr_stats(city, yr, False, cities)
        full_df = pd.concat([full_df, cur_df])
                
print(full_df.shape)

C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(1560, 47)


In [26]:
full_df['Median Household Income'] = full_df['Median Household Income'].str.lstrip('$')

In [27]:
full_df.to_csv('full_data.csv', index=False)

In [28]:
states = {
        'AK': 'Alaska', 'AL': 'Alabama',
        'AR': 'Arkansas', 'AZ': 'Arizona',
        'CA': 'California', 'CO': 'Colorado',
        'CT': 'Connecticut', 'DC': 'District of Columbia',
        'DE': 'Delaware', 'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii', 'IA': 'Iowa',
        'ID': 'Idaho', 'IL': 'Illinois',
        'IN': 'Indiana', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana',
        'MA': 'Massachusetts', 'MD': 'Maryland',
        'ME': 'Maine', 'MI': 'Michigan',
        'MN': 'Minnesota', 'MO': 'Missouri',
        'MS': 'Mississippi', 'MT': 'Montana',
        'NC': 'North Carolina', 'ND': 'North Dakota',
        'NE': 'Nebraska', 'NH': 'New Hampshire',
        'NJ': 'New Jersey', 'NM': 'New Mexico',
        'NV': 'Nevada', 'NY': 'New York',
        'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania',
        'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee',
        'TX': 'Texas', 'UT': 'Utah',
        'VA': 'Virginia', 'VT': 'Vermont',
        'WA': 'Washington', 'WI': 'Wisconsin',
        'WV': 'West Virginia', 'WY': 'Wyoming'
}

some_states = {'CA':'California', 'CO':'Colorado', 'FL':'Florida', 'MA':'Massachusetts','MN':'Minnesota', 'NY':'New York',
         'OH':'Ohio', 'TX':'Texas', 'WA':'Washington'}


In [29]:
test1 = city_yr_stats('AL', 2017, True, states)
state_df = pd.DataFrame(columns = test1.columns)

C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
yrs2 = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016
      ,2017]

for state in states:
    for yr in yrs2:
        cur_df = city_yr_stats(state, yr, True, states)
        state_df = pd.concat([state_df, cur_df])
                
print(state_df.shape)

C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(612, 44)


In [31]:
state_df['Median Household Income'] = state_df['Median Household Income'].str.lstrip('$')
state_df.head()

,Total Population,Area Total:,Area (Land),Area (Water),Total Population:,Less than 5 Years,Less than 10 Years,Less than 15 Years,Less than 18 Years,Less than 25 Years,...,Not a Citizen,State,year,gtax,gprice,HR,LR,CR,Bus,panelvar
0,"670,053","663,267.26",0.862,0.138,"670,053",0.071,0.145,0.217,0.268,0.38,...,0.037,AK,2006,0.08,2.591908,4600050.0,4600050.0,4600050.0,4600050.0,1
0,"683,478","663,267.26",0.862,0.138,"683,478",0.074,0.143,0.214,0.265,0.383,...,0.035,AK,2007,0.08,2.754354,4761546.0,4761546.0,4761546.0,4761546.0,1
0,"686,293","663,267.26",0.862,0.138,"686,293",0.075,0.146,0.217,0.263,0.381,...,0.031,AK,2008,0.08,3.590647,5079974.0,5079974.0,5079974.0,5079974.0,1
0,"698,473","663,267.26",0.862,0.138,"698,473",0.078,0.143,0.218,0.263,0.383,...,0.033,AK,2009,0.08,2.803689,5078129.0,5078129.0,5078129.0,5078129.0,1
0,"713,985","665,384.10",0.858,0.142,"713,985",0.075,0.145,0.218,0.264,0.369,...,0.035,AK,2010,0.08,3.366833,5089725.0,5089725.0,5089725.0,5089725.0,1


In [32]:
state_df.to_csv('state_controls.csv', index=False)

In [33]:
test2 = city_yr_stats('CA', 2017, True, states, True)
state_month_df = pd.DataFrame(columns = test2.columns)

for state in some_states:
    for yr in yrs2:
        cur_df = city_yr_stats(state, yr, True, states, True)
        state_month_df = pd.concat([state_month_df, cur_df])
                


C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
state_month_df['Median Household Income'] = state_month_df['Median Household Income'].str.lstrip('$')
state_month_df.to_csv('state_month_controls.csv', index=False)

In [35]:
test3 = city_yr_stats('CA', 2017, True, states, True, True)
state_mode_df = pd.DataFrame(columns = test3.columns)

for state in some_states:
    for yr in yrs2:
        cur_df = city_yr_stats(state, yr, True, states, True, True)
        state_mode_df = pd.concat([state_mode_df, cur_df])
        
state_mode_df['Median Household Income'] = state_mode_df['Median Household Income'].str.lstrip('$')
state_mode_df.to_csv('state_month_mode_controls.csv', index=False)

C:\Users\Jack Graham\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
